# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **Classification, because we have a discrete number (in this case 2) of possible outcomes that we are trying to predict. Regression is about mapping inputs to continuous outputs and classification is about mapping inputs with discrete outputs.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [33]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [41]:
# TODO: Calculate number of students
n_students = student_data.shape[0]

# TODO: Calculate number of features
n_features = student_data.shape[1]-1

# TODO: Calculate passing students
n_passed = student_data[student_data['passed']=='yes'].shape[0]

# TODO: Calculate failing students
n_failed = student_data[student_data['passed']=='no'].shape[0]

# TODO: Calculate graduation rate
grad_rate = n_passed/float(n_students)*100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [35]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [36]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
y_all=y_all.replace('yes',1)
y_all=y_all.replace('no',0)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [37]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split
# TODO: Set the number of training points
#num_train = 300

# Set the number of testing points
#num_test = X_all.shape[0] - num_train
#num_test = 95

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=95, random_state=42)
#X_train = None
#X_test = None
#y_train = None
#y_test = None

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. What are the general applications of each model? What are their strengths and weaknesses? Given what you know about the data, why did you choose these models to be applied?*

**Answer: ** The three supervised learning algorithms are SVM, decision trees and naive Bayes.

**SVM applications:** text categorization, image classification, character recognition

**SVM strengths:** ability to handle large feature spaces

**SVM weaknesses:**sensitive to noise, it only considers 2 classes


**decision tree applications:** finance, game theory

**decision tree strengths:** simple to understand and interpret, fast, implicitly performs feature selection

**decision tree weaknesses:**can easily overfit, problems with out-of-sample prediction


**naive bayes applications:** spam detection, sentiment analysis, face recognition, real time prediction, recommendation systems, multiclass prediction

**naive bayes strengths:**  fast to train and classify, relative small amount of test data needed, works well on categorical features, not sensitive to irrelevant features

**naive bayes weaknesses:** assumes independence of features which is seldom 100% the case


**Reason for SVM: ** SVM's are effective in high dimensional spaces. The dataset has a relative high number of features so SVM could be a good technique here.

**Reason for decision tree: ** Decision trees can work with different kinds of discrete feature data (numeric as well as categoric) because each value of the feature represents a branch in the tree. The data set of the student-data.csv file fits these criteria. The ID3 algorithm also selects the features with the highest information gain first. The data set has a high number of features (30) and maybe not all of them are relevant to predict the likelihood of a student to pass.

**Reason for naive bayes: ** The data set that was provided is relatively small: only 396 observations that we need to split into a training and a test set. The naieve Bayes algorithm converges quicker than some other classifiaction models and as a consequence can deal with small data sets.


### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [38]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    from time import time
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    from time import time
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    #return f1_score(target.values, y_pred, pos_label='yes')
    return f1_score(target.values, y_pred, pos_label=1)


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    from time import time
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [39]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A
# from sklearn import model_B
# from skearln import model_C
#http://stackoverflow.com/questions/11886862/calculating-computational-time-and-memory-for-a-code-in-python
import time
from sklearn.metrics import f1_score
from sklearn.svm import SVC
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

# TODO: Initialize the three models
clf_A = SVC(kernel="linear")
clf_B = tree.DecisionTreeClassifier()
#clf_C = GaussianNB()
clf_C = MultinomialNB(alpha=1.0)

# TODO: Set up the training set sizes
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train
y_train_300 = y_train

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)


def runModel(clf):
    time_start=time.clock()
    clf.fit(X_train_100,y_train_100)
    trainTime_100=(time.clock() - time_start)
    print("train time 100 :"+str(trainTime_100))
    time_start=time.clock()
    pred_100=clf.predict(X_test)
    f1_score_100_test=f1_score(y_test, pred_100, average='micro')  
    f1_score_100_train=f1_score(y_train,clf.predict(X_train),average='micro')  
    predTime_100=(time.clock() - time_start)
    print("prediction time 100 :"+str(predTime_100))
    print("F1 score 100 test :"+str(f1_score_100_test))
    print("F1 score 100 train :"+str(f1_score_100_train))
    print(" ")

    time_start=time.clock()
    clf.fit(X_train_200,y_train_200)
    trainTime_200=(time.clock() - time_start)
    print("train time 200 :"+str(trainTime_200))
    time_start=time.clock()
    pred_200=clf.predict(X_test)
    f1_score_200_test=f1_score(y_test, pred_200, average='micro')   
    f1_score_200_train=f1_score(y_train,clf.predict(X_train),average='micro')  
    predTime_200=(time.clock() - time_start)
    print("prediction time 200 :"+str(predTime_200))
    print("F1 score 200 test:"+str(f1_score_200_test))
    print("F1 score 200 train :"+str(f1_score_200_train))
    print(" ")
    
    time_start=time.clock()
    clf.fit(X_train_300,y_train_300)
    trainTime_300=(time.clock() - time_start)
    print("train time 300 :"+str(trainTime_300))
    time_start=time.clock()
    pred_300=clf.predict(X_test)
    f1_score_300_test=f1_score(y_test, pred_300, average='micro')
    f1_score_300_train=f1_score(y_train,clf.predict(X_train),average='micro')  
    predTime_300=(time.clock() - time_start)
    print("prediction time 300 :"+str(predTime_300))
    print("F1 score 300 test:"+str(f1_score_300_test))
    print("F1 score 300 train :"+str(f1_score_300_train))
    print(" ")
    
print('SVM')
runModel(clf_A)
print('')
print('decision tree')
runModel(clf_B)
print('')
print('naive bayes')
runModel(clf_C)

SVM
train time 100 :0.00427352752013
prediction time 100 :0.00277339296008
F1 score 100 test :0.772727272727
F1 score 100 train :0.80195599022
 
train time 200 :0.0209660444234
prediction time 200 :0.00334106333321
F1 score 200 test:0.768115942029
F1 score 200 train :0.817351598174
 
train time 300 :0.024866778258
prediction time 300 :0.00388697406743
F1 score 300 test:0.781954887218
F1 score 300 train :0.854586129754
 

decision tree
train time 100 :0.000784306699643
prediction time 100 :0.00108990151784
F1 score 100 test :0.610169491525
F1 score 100 train :0.773067331671
 
train time 200 :0.0014047761797
prediction time 200 :0.0010931014549
F1 score 200 test:0.744186046512
F1 score 200 train :0.888337468983
 
train time 300 :0.00175005031633
prediction time 300 :0.00108958152123
F1 score 300 test:0.638655462185
F1 score 300 train :1.0
 

naive bayes
train time 100 :0.000592949945712
prediction time 100 :0.00149117471301
F1 score 100 test :0.753846153846
F1 score 100 train :0.74216867

C:\Users\Jeroen\Anaconda\lib\site-packages\sklearn\metrics\classification.py:931: DeprecationWarning: From version 0.18, binary input will not be handled specially when using averaged precision/recall/F-score. Please use average='binary' to report only the positive class performance.
  'positive class performance.', DeprecationWarning)
C:\Users\Jeroen\Anaconda\lib\site-packages\sklearn\metrics\classification.py:931: DeprecationWarning: From version 0.18, binary input will not be handled specially when using averaged precision/recall/F-score. Please use average='binary' to report only the positive class performance.
  'positive class performance.', DeprecationWarning)
C:\Users\Jeroen\Anaconda\lib\site-packages\sklearn\metrics\classification.py:931: DeprecationWarning: From version 0.18, binary input will not be handled specially when using averaged precision/recall/F-score. Please use average='binary' to report only the positive class performance.
  'positive class performance.', Deprec

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - SVM**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.00425944775634        |  0.00277339296008      |     0.797        |    0.772        |
| 200               | 0.0198227038054         |  0.00308826762193      |     0.817        |    0.768        |
| 300               | 0.0246658216638         |  0.00402489173575      |     0.854        |    0.781        |

** Classifer 2 - decision tree**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.000792306571384       |  0.00108926152461      |    0.782         |     0.621       |
| 200               | 0.0012521387689         |  0.00107614175067      |    0.896         |     0.754       |
| 300               | 0.0017298906605         |  0.00108222164272      |    1.0           |     0.694       |

** Classifer 3 - naive bayes**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |  0.000577270198846      |  0.00145661529677      |   0.742          |   0.754         |
| 200               |  0.000803826376796      |  0.00146269520337      |   0.791          |   0.794         |
| 300               |  0.000759667120292      |  0.00154621376714      |   0.795          |   0.794         |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Chosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **I chose naieve bayes because it is the fastest model to train and needs the smallest training set to achieve good and stable F1 scores. The advantage of taking naieve base is that you don't need to invest in computing power as much as in the case of SVM and DT. Also the fact that it needs a small training set means that you need less memory on your computer system. Another advantage of naive base is that it is easy/fast to retrain the model when new data is available.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. For example if you've chosen to use a decision tree or a support vector machine, how does the model go about making a prediction?*

**Answer: **The naive bayes algorithm will calculate the probability that a student will pass given this student's age, health, family size etc. The algorithm will be trained on a set of data called the training set. 

We will start with a prior probability that a student will pass. This is the passing rate of a student without knowing any further details of that student. 

In a second step we calculate the posterior probability for each feature of the student (age,...). For example for the feature "age" we calculate the probability of passing given the age of the student.

Finally we will combine (via the naive Bayes equation) the prior and posterior probabilities to calculate the probability that a particular student will pass. In other words we use the information of the features of a student to calculate the chance of passing for a specific student.

Once we have calculated the prior and posterior probabilities, we can apply these on the features of a new student and predict the likelihood of this student to pass.

After a period of time we will have more data of new students who passed. We can use this data to make the model better by re-calculating the prior and posterior probabilities with this new data.


### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [40]:
# TODO: Import 'gridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer, f1_score

# TODO: Create the parameters list you wish to tune
parameters = {'alpha':range(100), 'fit_prior':('True','False')}
#parameters = {'alpha':range(10)}

# TODO: Initialize the classifier
clf = MultinomialNB()

# TODO: Make an f1 scoring function using 'make_scorer' 
#f1_scorer = make_scorer(predict_labels)
f1_scorer = make_scorer(f1_score, pos_label=1)

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
#grid_obj = GridSearchCV(clf, parameters)
grid_obj = GridSearchCV(clf, parameters,scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj.fit(X_train,y_train)

# Get the estimator
clf = grid_obj.best_estimator_
print(clf)

# Report the final F1 score for training and testing after parameter tuning
#print(X_train)
#print(y_train)
#print(X_test)
#print(y_test)
#print(parameters)
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

MultinomialNB(alpha=14, class_prior=None, fit_prior='True')
Made predictions in 0.0000 seconds.
Tuned model has a training F1 score of 0.8079.
Made predictions in 0.0010 seconds.
Tuned model has a testing F1 score of 0.8000.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: ** The initial model's alpha was 1 and this gave a training F1 score of 0.8009 and a testing F1 score of 0.7972.
We tuned the model for alpha and the optimal alpha is 12. But this only marginally improved the F1 scores to 0.8079 and 0.8000

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.